In [3]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import sklearn.linear_model as lm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In order to make the dataset, the raw data must be in a directory 'Not_Zip' and named with its state name, e.g. 'Not_Zip/AZ.csv' for Arizona

### States to keep:

Republicans: 
1. Arizona (3.5 million)
2. Texas (27 million)

Democrats:
1. Illinois (13 million)
2. Washington (11 million)

### Features to take:
1. Date
2. Race
3. Sex

Take 500 000 datapoint per set -> we end with a dataset with roughly 2 million datapoints

In [4]:
PATH = 'Not_Zip/'
cols_to_use = ['date','subject_race', 'subject_sex', 'search_conducted']
States = ['AZ', 'TX', 'IL', 'WA']
number_subsamples = 500000

In [5]:
# Get the biggest time range for which all datasets contains values
min_date = dt.datetime(1900,1,1)
max_date = dt.datetime(2030,1,1)

for state in States:
    
    df = pd.read_csv(PATH + state + '.csv', usecols = cols_to_use)
    df.dropna(inplace = True)
    df.date = pd.to_datetime(df.date)
    df_min_date = df.date.min()
    df_max_date = df.date.max()
    
    if df_min_date > min_date:
        min_date = df_min_date
    
    if df_max_date < max_date:
        max_date = df_max_date

print('Min_date = {}, Max_date = {}'.format(min_date, max_date))    

Min_date = 2012-01-01 00:00:00, Max_date = 2016-12-31 00:00:00


In [6]:
df = pd.DataFrame([], columns = ['date', 'subject_race', 'subject_sex', 'search_conducted', 'State', 'Partisanship'])
republicans = ['TX', 'AZ']
for state in States:
    
    # Get the dataset with only the desired columns
    df_state = pd.read_csv(PATH + state + '.csv', usecols = cols_to_use)
    
    # Drop nan values
    df_state.dropna(inplace = True)
    
    # Only keep Hispanic, Black and White drivers
    df_state = df_state.iloc[[x in ['hispanic', 'black', 'white'] for x in df_state.subject_race]]
    
    # Keep only data in the good time range
    df_state.date = pd.to_datetime(df_state.date)
    df_state = df_state[(df_state.date > min_date) & (df_state.date < max_date)]
    
    # Randomly create indices of the datapoints to keep
    inds = np.random.permutation(range(df_state.shape[0]))[:number_subsamples]
    
    # Keep only the subsample
    df_state = df_state.iloc[inds]
    
    # Append the state so that we know where the datapoints come from
    df_state['State'] = df_state.date.apply(lambda x: state)
    df_state['Partisanship'] = df_state.State.apply(lambda x: 'R' if x in republicans else 'D')
    
    # Concatenate with the final dataset
    df = pd.concat((df, df_state))
    
    del df_state
    
    print('State {} processed.'.format(state))

State AZ processed.
State TX processed.
State IL processed.
State WA processed.


In [7]:
df.to_csv('data_time.csv', index = False)

In [8]:
df = pd.read_csv('data.csv')

In [9]:
df.head()

,date,subject_race,subject_sex,search_conducted,State,Partisanship
0,2014-02-05,white,male,False,AZ,R
1,2014-01-31,hispanic,male,False,AZ,R
2,2014-02-02,white,female,False,AZ,R
3,2014-05-25,hispanic,male,False,AZ,R
4,2015-05-01,white,male,False,AZ,R
